In [1]:
# Importing libraries and setting up the environment
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from statsmodels.genmod.families import Binomial
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from mpl_toolkits.basemap import Basemap
import matplotlib.ticker as plticker
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")  # setting ignore as a parameter

In [2]:
#  Predefined Code
# Function to read variable in data
def process_climate(filepath):
    variable_data = pd.read_csv(
        filepath,
        names=[
            "POTVEG",
            "YEAR",
            "NGRID",
            "TOTCELLAREA",
            "TOTFORECOZONE",
            "MXPRED",
            "MNPRED",
            "MNBYAR",
            "SIMPMN",
            "STNDEV",
            "MNTOTYR",
        ],
    )

    return variable_data


# Function to process historical data
def process_dataset(filepath, dataset_name):
    # Import historical Data
    historical_data = process_climate(filepath)

    # Select Total For All PFTs (historical)
    total_historical_yearly_pft = historical_data.query(" POTVEG==99 and YEAR >= 1984")

    # Select last 30 Years for historical
    historical_data = historical_data.query("YEAR >= 1984 and POTVEG != 99")
    historical_data = historical_data[["POTVEG", "YEAR", "TOTFORECOZONE"]]

    # Rename Total Column
    historical_data.rename(
        columns={"TOTFORECOZONE": f"TOTAL{dataset_name}"},
        inplace=True,
    )

    return historical_data

In [3]:
# Import end_century data
end_century_vegc = pd.read_csv("../data/vegc_agg_models.csv", header=[0]).query("YEAR >=2069")
end_century_npp = pd.read_csv("../data/npp_agg_models.csv", header=[0]).query("YEAR >=2069")
end_century_nep = pd.read_csv("../data/nep_agg_models.csv", header=[0]).query("YEAR >=2069")
end_century_soilorgc = pd.read_csv("../data/soilorgc_agg_models.csv", header=[0]).query("YEAR >=2069")
end_century_availn = pd.read_csv("../data/availn_agg_models.csv", header=[0]).query("YEAR >=2069")
end_century_netnmin = pd.read_csv("../data/netnmin_agg_models.csv", header=[0]).query("YEAR >=2069")

In [4]:
# Import historical data
historical_vegc = process_dataset("../data/historical/ORIGINAL_LULC_VEGC.SUMMARY", "VEGC")
historical_npp = process_dataset("../data/historical/ORIGINAL_LULC_NPP.SUMMARY", "NPP")
historical_nep = process_dataset("../data/historical/ORIGINAL_LULC_NEP.SUMMARY", "NEP")
historical_soilorgc = process_dataset("../data/historical/ORIGINAL_LULC_SOILORGC.SUMMARY", "SOILORGC")
historical_availn = process_dataset("../data/historical/ORIGINAL_LULC_AVAILN.SUMMARY", "AVAILN")
historical_netnmin = process_dataset("../data/historical/ORIGINAL_LULC_NETNMIN.SUMMARY", "NETNMIN")

In [5]:
# PFT Specific Future Data
BF_end_century_vegc = end_century_vegc.query("POTVEG == 4")
MTF_end_century_vegc = end_century_vegc.query("POTVEG == 8")
TCF_end_century_vegc = end_century_vegc.query("POTVEG == 9")
TDF_end_century_vegc = end_century_vegc.query("POTVEG == 10")
SG_end_century_vegc = end_century_vegc.query("POTVEG == 13")
AS_end_century_vegc = end_century_vegc.query("POTVEG == 15")
XFW_end_century_vegc = end_century_vegc.query("POTVEG == 19")
TBEF_end_century_vegc = end_century_vegc.query("POTVEG == 33")

# PFT Specific historical Data
BF_historical_vegc = historical_vegc.query("POTVEG== 4")
MTF_historical_vegc = historical_vegc.query("POTVEG== 8")
TCF_historical_vegc = historical_vegc.query("POTVEG== 9")
TDF_historical_vegc = historical_vegc.query("POTVEG== 10")
SG_historical_vegc = historical_vegc.query("POTVEG== 13")
AS_historical_vegc = historical_vegc.query("POTVEG== 15")
XFW_historical_vegc = historical_vegc.query("POTVEG== 19")
TBEF_historical_vegc = historical_vegc.query("POTVEG== 33")

In [6]:
# TTEST VEGC
def do_ttest(historical_vegc, end_century_vegc):
    # Truncate the longer array to the length of the shorter array
    min_length = min(len(historical_vegc), len(end_century_vegc))
    period_1984_2014 = historical_vegc[:min_length]
    period_2069_2099 = end_century_vegc[:min_length]

    # Convert the lists to numpy arrays
    period_1984_2014 = np.array(period_1984_2014)
    period_2069_2099 = np.array(period_2069_2099)

    tscore, pvalue = stats.ttest_rel(period_1984_2014, period_2069_2099)
    return tscore, pvalue


# Initialize a dictionary to store the t-scores and p-values for each group
results = {}

groups = ["BF", "MTF", "TCF", "TDF", "SG", "AS", "XFW", "TBEF"]

# Iterate through each group and calculate the t-score and p-value
for group in groups:
    historical_vegc = globals()[group + "_historical_vegc"]["TOTALVEGC"]
    end_century_vegc = globals()[group + "_end_century_vegc"]["TOTALVEGC"]
    tscore, pvalue = do_ttest(historical_vegc, end_century_vegc)

    # Add the t-score and p-value to the results dictionary
    results[group] = {"tscore": tscore, "pvalue": pvalue, "significant": pvalue < 0.05}


# data = [{"pft": group, "t statistic": result["tscore"], "p value": result["pvalue"], "significant difference": result["significant"]} for group, result in results.items()]

# Create a list of dictionaries with the results data
data = [
    {
        "pft": group,
        "t statistic": result["tscore"],
        "p value": result["pvalue"],
        "significant difference": result["significant"],
    }
    for group, result in results.items()
]

# Create a DataFrame from the list of dictionaries
statistics = pd.DataFrame(data)

# Reorder the columns
statistics = statistics[["pft", "t statistic", "p value", "significant difference"]]
statistics=statistics.round(6)

statistics.to_csv("../data/total_stats.csv", index=False)
# Print the DataFrame
statistics.head(10)

,pft,t statistic,p value,significant difference
0,BF,50.748028,0.000000,True
1,MTF,24.538948,0.000000,True
2,TCF,53.351585,0.000000,True
3,TDF,0.179522,0.886917,False
4,SG,NaN,NaN,False
5,AS,NaN,NaN,False
6,XFW,11.224175,0.000000,True
7,TBEF,-2.654714,0.012579,True


In [7]:
# PFT Specific Future Data
BF_end_century_npp = end_century_npp.query("POTVEG == 4")
MTF_end_century_npp = end_century_npp.query("POTVEG == 8")
TCF_end_century_npp = end_century_npp.query("POTVEG == 9")
TDF_end_century_npp = end_century_npp.query("POTVEG == 10")
SG_end_century_npp = end_century_npp.query("POTVEG == 13")
AS_end_century_npp = end_century_npp.query("POTVEG == 15")
XFW_end_century_npp = end_century_npp.query("POTVEG == 19")
TBEF_end_century_npp = end_century_npp.query("POTVEG == 33")

# PFT Specific historical Data
BF_historical_npp = historical_npp.query("POTVEG== 4")
MTF_historical_npp = historical_npp.query("POTVEG== 8")
TCF_historical_npp = historical_npp.query("POTVEG== 9")
TDF_historical_npp = historical_npp.query("POTVEG== 10")
SG_historical_npp = historical_npp.query("POTVEG== 13")
AS_historical_npp = historical_npp.query("POTVEG== 15")
XFW_historical_npp = historical_npp.query("POTVEG== 19")
TBEF_historical_npp = historical_npp.query("POTVEG== 33")


In [8]:
# TTEST NPP
def do_ttest(historical_npp, end_century_npp):
    # Truncate the longer array to the length of the shorter array
    min_length = min(len(historical_npp), len(end_century_npp))
    period_1984_2014 = historical_npp[:min_length]
    period_2069_2099 = end_century_npp[:min_length]

    # Convert the lists to numpy arrays
    period_1984_2014 = np.array(period_1984_2014)
    period_2069_2099 = np.array(period_2069_2099)

    tscore, pvalue = stats.ttest_rel(period_1984_2014, period_2069_2099)
    return tscore, pvalue


# Initialize a dictionary to store the t-scores and p-values for each group
results = {}

groups = ["BF", "MTF", "TCF", "TDF", "SG", "AS", "XFW", "TBEF"]

# Iterate through each group and calculate the t-score and p-value
for group in groups:
    historical_npp = globals()[group + "_historical_npp"]["TOTALNPP"]
    end_century_npp = globals()[group + "_end_century_npp"]["TOTALNPP"]
    tscore, pvalue = do_ttest(historical_npp, end_century_npp)

    # Add the t-score and p-value to the results dictionary
    results[group] = {"tscore": tscore, "pvalue": pvalue, "significant": pvalue < 0.05}


# data = [{"pft": group, "t statistic": result["tscore"], "p value": result["pvalue"], "significant difference": result["significant"]} for group, result in results.items()]

# Create a list of dictionaries with the results data
data = [
    {
        "pft": group,
        "t statistic": result["tscore"],
        "p value": result["pvalue"],
        "significant difference": result["significant"],
    }
    for group, result in results.items()
]

# Create a DataFrame from the list of dictionaries
statistics = pd.DataFrame(data)

# Reorder the columns
statistics = statistics[["pft", "t statistic", "p value", "significant difference"]]
statistics=statistics.round(6)
statistics.to_csv("../data/total_stats.csv", index=False, mode="a")
# Print the DataFrame
statistics.head(10)

,pft,t statistic,p value,significant difference
0,BF,17.143430,0.000000,True
1,MTF,14.614482,0.000000,True
2,TCF,30.436335,0.000000,True
3,TDF,-0.555392,0.677251,False
4,SG,NaN,NaN,False
5,AS,NaN,NaN,False
6,XFW,4.753492,0.000047,True
7,TBEF,-0.655583,0.517089,False


In [9]:
## PFT Specific Future Data
BF_end_century_nep= end_century_nep.query("POTVEG == 4")
MTF_end_century_nep= end_century_nep.query("POTVEG == 8")
TCF_end_century_nep= end_century_nep.query("POTVEG == 9")
TDF_end_century_nep= end_century_nep.query("POTVEG == 10")
SG_end_century_nep= end_century_nep.query("POTVEG == 13")
AS_end_century_nep= end_century_nep.query("POTVEG == 15")
XFW_end_century_nep= end_century_nep.query("POTVEG == 19")
TBEF_end_century_nep= end_century_nep.query("POTVEG == 33")

# PFT Specific historical Data
BF_historical_nep= historical_nep.query("POTVEG== 4")
MTF_historical_nep= historical_nep.query("POTVEG== 8")
TCF_historical_nep= historical_nep.query("POTVEG== 9")
TDF_historical_nep= historical_nep.query("POTVEG== 10")
SG_historical_nep= historical_nep.query("POTVEG== 13")
AS_historical_nep= historical_nep.query("POTVEG== 15")
XFW_historical_nep= historical_nep.query("POTVEG== 19")
TBEF_historical_nep= historical_nep.query("POTVEG== 33")

In [10]:
# TTEST NEP
def do_ttest(historical_nep, end_century_nep):
    # Truncate the longer array to the length of the shorter array
    min_length = min(len(historical_nep), len(end_century_nep))
    period_1984_2014 = historical_nep[:min_length]
    period_2069_2099 = end_century_nep[:min_length]

    # Convert the lists to numpy arrays
    period_1984_2014 = np.array(period_1984_2014)
    period_2069_2099 = np.array(period_2069_2099)

    tscore, pvalue = stats.ttest_rel(period_1984_2014, period_2069_2099)
    return tscore, pvalue


# Initialize a dictionary to store the t-scores and p-values for each group
results = {}

groups = ["BF", "MTF", "TCF", "TDF", "SG", "AS", "XFW", "TBEF"]

# Iterate through each group and calculate the t-score and p-value
for group in groups:
    historical_nep = globals()[group + "_historical_nep"]["TOTALNEP"]
    end_century_nep = globals()[group + "_end_century_nep"]["TOTALNEP"]
    tscore, pvalue = do_ttest(historical_nep, end_century_nep)

    # Add the t-score and p-value to the results dictionary
    results[group] = {"tscore": tscore, "pvalue": pvalue, "significant": pvalue < 0.05}


# data = [{"pft": group, "t statistic": result["tscore"], "p value": result["pvalue"], "significant difference": result["significant"]} for group, result in results.items()]

# Create a list of dictionaries with the results data
data = [
    {
        "pft": group,
        "t statistic": result["tscore"],
        "p value": result["pvalue"],
        "significant difference": result["significant"],
    }
    for group, result in results.items()
]

# Create a DataFrame from the list of dictionaries
statistics = pd.DataFrame(data)

# Reorder the columns
statistics = statistics[["pft", "t statistic", "p value", "significant difference"]]
statistics=statistics.round(6)
statistics.to_csv("../data/total_stats.csv", index=False, mode="a")
statistics.head(10)

,pft,t statistic,p value,significant difference
0,BF,-2.383271,0.023692,True
1,MTF,-0.026726,0.978855,False
2,TCF,-0.252739,0.802194,False
3,TDF,-0.847237,0.552528,False
4,SG,NaN,NaN,False
5,AS,NaN,NaN,False
6,XFW,-1.034092,0.309358,False
7,TBEF,-0.925733,0.361972,False


In [11]:
# PFT Specific Future Data
BF_end_century_soilorgc = end_century_soilorgc.query("POTVEG == 4")
MTF_end_century_soilorgc = end_century_soilorgc.query("POTVEG == 8")
TCF_end_century_soilorgc = end_century_soilorgc.query("POTVEG == 9")
TDF_end_century_soilorgc = end_century_soilorgc.query("POTVEG == 10")
SG_end_century_soilorgc = end_century_soilorgc.query("POTVEG == 13")
AS_end_century_soilorgc = end_century_soilorgc.query("POTVEG == 15")
XFW_end_century_soilorgc = end_century_soilorgc.query("POTVEG == 19")
TBEF_end_century_soilorgc = end_century_soilorgc.query("POTVEG == 33")

# PFT Specific historical Data
BF_historical_soilorgc = historical_soilorgc.query("POTVEG== 4")
MTF_historical_soilorgc = historical_soilorgc.query("POTVEG== 8")
TCF_historical_soilorgc = historical_soilorgc.query("POTVEG== 9")
TDF_historical_soilorgc = historical_soilorgc.query("POTVEG== 10")
SG_historical_soilorgc = historical_soilorgc.query("POTVEG== 13")
AS_historical_soilorgc = historical_soilorgc.query("POTVEG== 15")
XFW_historical_soilorgc = historical_soilorgc.query("POTVEG== 19")
TBEF_historical_soilorgc = historical_soilorgc.query("POTVEG== 33")




In [12]:
# TTEST soilorgc
def do_ttest(historical_soilorgc, end_century_soilorgc):
    # Truncate the longer array to the length of the shorter array
    min_length = min(len(historical_soilorgc), len(end_century_soilorgc))
    period_1984_2014 = historical_soilorgc[:min_length]
    period_2069_2099 = end_century_soilorgc[:min_length]

    # Convert the lists to numpy arrays
    period_1984_2014 = np.array(period_1984_2014)
    period_2069_2099 = np.array(period_2069_2099)

    tscore, pvalue = stats.ttest_rel(period_1984_2014, period_2069_2099)
    return tscore, pvalue


# Initialize a dictionary to store the t-scores and p-values for each group
results = {}

groups = ["BF", "MTF", "TCF", "TDF", "SG", "AS", "XFW", "TBEF"]

# Iterate through each group and calculate the t-score and p-value
for group in groups:
    historical_soilorgc = globals()[group + "_historical_soilorgc"]["TOTALSOILORGC"]
    end_century_soilorgc = globals()[group + "_end_century_soilorgc"]["TOTALSOILORGC"]
    tscore, pvalue = do_ttest(historical_soilorgc, end_century_soilorgc)

    # Add the t-score and p-value to the results dictionary
    results[group] = {"tscore": tscore, "pvalue": pvalue, "significant": pvalue < 0.05}


# data = [{"pft": group, "t statistic": result["tscore"], "p value": result["pvalue"], "significant difference": result["significant"]} for group, result in results.items()]

# Create a list of dictionaries with the results data
data = [
    {
        "pft": group,
        "t statistic": result["tscore"],
        "p value": result["pvalue"],
        "significant difference": result["significant"],
    }
    for group, result in results.items()
]

# Create a DataFrame from the list of dictionaries
statistics = pd.DataFrame(data)

# Reorder the columns
statistics = statistics[["pft", "t statistic", "p value", "significant difference"]]
statistics=statistics.round(6)
statistics.to_csv("../data/total_stats.csv", index=False, mode="a")
# Print the DataFrame
statistics.head(10)

,pft,t statistic,p value,significant difference
0,BF,63.111889,0.00000,True
1,MTF,38.518013,0.00000,True
2,TCF,116.631655,0.00000,True
3,TDF,0.921220,0.52609,False
4,SG,NaN,NaN,False
5,AS,NaN,NaN,False
6,XFW,71.921108,0.00000,True
7,TBEF,8.631902,0.00000,True


In [13]:
# PFT Specific Future Data
BF_end_century_netnmin = end_century_netnmin.query("POTVEG == 4")
MTF_end_century_netnmin = end_century_netnmin.query("POTVEG == 8")
TCF_end_century_netnmin = end_century_netnmin.query("POTVEG == 9")
TDF_end_century_netnmin = end_century_netnmin.query("POTVEG == 10")
SG_end_century_netnmin = end_century_netnmin.query("POTVEG == 13")
AS_end_century_netnmin = end_century_netnmin.query("POTVEG == 15")
XFW_end_century_netnmin = end_century_netnmin.query("POTVEG == 19")
TBEF_end_century_netnmin = end_century_netnmin.query("POTVEG == 33")

# PFT Specific historical Data
BF_historical_netnmin = historical_netnmin.query("POTVEG== 4")
MTF_historical_netnmin = historical_netnmin.query("POTVEG== 8")
TCF_historical_netnmin = historical_netnmin.query("POTVEG== 9")
TDF_historical_netnmin = historical_netnmin.query("POTVEG== 10")
SG_historical_netnmin = historical_netnmin.query("POTVEG== 13")
AS_historical_netnmin = historical_netnmin.query("POTVEG== 15")
XFW_historical_netnmin = historical_netnmin.query("POTVEG== 19")
TBEF_historical_netnmin = historical_netnmin.query("POTVEG== 33")




In [14]:
# TTEST netnmin
def do_ttest(historical_netnmin, end_century_netnmin):
    # Truncate the longer array to the length of the shorter array
    min_length = min(len(historical_netnmin), len(end_century_netnmin))
    period_1984_2014 = historical_netnmin[:min_length]
    period_2069_2099 = end_century_netnmin[:min_length]

    # Convert the lists to numpy arrays
    period_1984_2014 = np.array(period_1984_2014)
    period_2069_2099 = np.array(period_2069_2099)

    tscore, pvalue = stats.ttest_rel(period_1984_2014, period_2069_2099)
    return tscore, pvalue


# Initialize a dictionary to store the t-scores and p-values for each group
results = {}

groups = ["BF", "MTF", "TCF", "TDF", "SG", "AS", "XFW", "TBEF"]

# Iterate through each group and calculate the t-score and p-value
for group in groups:
    historical_netnmin = globals()[group + "_historical_netnmin"]["TOTALNETNMIN"]
    end_century_netnmin = globals()[group + "_end_century_netnmin"]["TOTALNETNMIN"]
    tscore, pvalue = do_ttest(historical_netnmin, end_century_netnmin)

    # Add the t-score and p-value to the results dictionary
    results[group] = {"tscore": tscore, "pvalue": pvalue, "significant": pvalue < 0.05}


# data = [{"pft": group, "t statistic": result["tscore"], "p value": result["pvalue"], "significant difference": result["significant"]} for group, result in results.items()]

# Create a list of dictionaries with the results data
data = [
    {
        "pft": group,
        "t statistic": result["tscore"],
        "p value": result["pvalue"],
        "significant difference": result["significant"],
    }
    for group, result in results.items()
]

# Create a DataFrame from the list of dictionaries
statistics = pd.DataFrame(data)

# Reorder the columns
statistics = statistics[["pft", "t statistic", "p value", "significant difference"]]
statistics=statistics.round(6)
statistics.to_csv("../data/total_stats.csv", index=False, mode="a")
# Print the DataFrame
statistics.head(10)

,pft,t statistic,p value,significant difference
0,BF,26.823483,0.000000,True
1,MTF,25.461265,0.000000,True
2,TCF,47.362496,0.000000,True
3,TDF,1.884593,0.310569,False
4,SG,NaN,NaN,False
5,AS,NaN,NaN,False
6,XFW,3.145061,0.003730,True
7,TBEF,3.806445,0.000648,True


In [15]:
# PFT Specific Future Data
BF_end_century_availn = end_century_availn.query("POTVEG == 4")
MTF_end_century_availn = end_century_availn.query("POTVEG == 8")
TCF_end_century_availn = end_century_availn.query("POTVEG == 9")
TDF_end_century_availn = end_century_availn.query("POTVEG == 10")
SG_end_century_availn = end_century_availn.query("POTVEG == 13")
AS_end_century_availn = end_century_availn.query("POTVEG == 15")
XFW_end_century_availn = end_century_availn.query("POTVEG == 19")
TBEF_end_century_availn = end_century_availn.query("POTVEG == 33")

# PFT Specific historical Data
BF_historical_availn = historical_availn.query("POTVEG== 4")
MTF_historical_availn = historical_availn.query("POTVEG== 8")
TCF_historical_availn = historical_availn.query("POTVEG== 9")
TDF_historical_availn = historical_availn.query("POTVEG== 10")
SG_historical_availn = historical_availn.query("POTVEG== 13")
AS_historical_availn = historical_availn.query("POTVEG== 15")
XFW_historical_availn = historical_availn.query("POTVEG== 19")
TBEF_historical_availn = historical_availn.query("POTVEG== 33")

In [16]:
# TTEST availn
def do_ttest(historical_availn, end_century_availn):
    # Truncate the longer array to the length of the shorter array
    min_length = min(len(historical_availn), len(end_century_availn))
    period_1984_2014 = historical_availn[:min_length]
    period_2069_2099 = end_century_availn[:min_length]

    # Convert the lists to numpy arrays
    period_1984_2014 = np.array(period_1984_2014)
    period_2069_2099 = np.array(period_2069_2099)

    tscore, pvalue = stats.ttest_rel(period_1984_2014, period_2069_2099)
    return tscore, pvalue


# Initialize a dictionary to store the t-scores and p-values for each group
results = {}

groups = ["BF", "MTF", "TCF", "TDF", "SG", "AS", "XFW", "TBEF"]

# Iterate through each group and calculate the t-score and p-value
for group in groups:
    historical_availn = globals()[group + "_historical_availn"]["TOTALAVAILN"]
    end_century_availn = globals()[group + "_end_century_availn"]["TOTALAVAILN"]
    tscore, pvalue = do_ttest(historical_availn, end_century_availn)

    # Add the t-score and p-value to the results dictionary
    results[group] = {"tscore": tscore, "pvalue": pvalue, "significant": pvalue < 0.05}


# data = [{"pft": group, "t statistic": result["tscore"], "p value": result["pvalue"], "significant difference": result["significant"]} for group, result in results.items()]

# Create a list of dictionaries with the results data
data = [
    {
        "pft": group,
        "t statistic": result["tscore"],
        "p value": result["pvalue"],
        "significant difference": result["significant"],
    }
    for group, result in results.items()
]

# Create a DataFrame from the list of dictionaries
statistics = pd.DataFrame(data)

# Reorder the columns
statistics = statistics[["pft", "t statistic", "p value", "significant difference"]]
statistics=statistics.round(6)
statistics.to_csv("../data/total_stats.csv", index=False, mode="a")
# Print the DataFrame
statistics.head(10)

,pft,t statistic,p value,significant difference
0,BF,13.454800,0.000000,True
1,MTF,75.013603,0.000000,True
2,TCF,47.952151,0.000000,True
3,TDF,14.637891,0.043424,True
4,SG,NaN,NaN,False
5,AS,NaN,NaN,False
6,XFW,225.585061,0.000000,True
7,TBEF,69.162905,0.000000,True
